In [37]:
import pandas as pd
from collections import Counter
import nltk
import matplotlib
import matplotlib.pyplot as plt

file = pd.read_csv("Direct Messages - Group - 353 [1032844773272338482].csv")
file.drop(["AuthorID", "Attachments", "Reactions"], inplace=True, axis = 1)
file.dropna(subset=["Content"], inplace=True)
file.loc[:,"Author"] = file.loc[:,"Author"].str[:-5]
file

,Author,Date,Content
0,Zakoza,10/20/2022 7:36 PM,Changed the channel name: 353
2,Zakoza,10/20/2022 7:36 PM,Is this what your ab_analysis.py file outputs?
4,Dragonite,10/20/2022 8:28 PM,ur numbers seem right
5,Zakoza,10/20/2022 8:29 PM,how
6,Dragonite,10/20/2022 8:29 PM,idk my numbers are so small
...,...,...,...
285,Zakoza,12/14/2022 12:30 PM,gg
286,Zakoza,12/14/2022 12:31 PM,i had 14.9% out of the 26% through quiz 1 and 2
287,Zakoza,12/14/2022 12:31 PM,so im good
288,Parmesan,12/14/2022 12:44 PM,https://tenor.com/view/no-yes-ok-yes-no-maybe-...


In [40]:
stopwords = nltk.corpus.stopwords.words('english')
# RegEx for stopwords
RE_stopwords = r'\b(?:{})\b'.format('|'.join(stopwords))
# replace '|'-->' ' and drop all stopwords
words = (file.Content
           .str.lower()
           .replace([r'\'|=|\>|\<|u|r|\"|\:|\,|\.', r'\|', RE_stopwords], ['', ' ', ''], regex=True)
           .str.cat(sep=' ')
           .split()
)

# generate DF out of Counter
rslt = pd.DataFrame(Counter(words).most_common(50),
                    columns=['Word', 'Frequency']).set_index('Word')
print(rslt)

         Frequency
Word              
yo              20
fo              20
ae              18
se              18
jst             16
im              11
get              9
got              9
gys              9
know             8
think            8
pt               8
?                8
like             7
yeah             7
done             7
qiz              7
idk              6
abot             6
one              6
cz               6
said             6
353              5
ight             5
oh               5
thogh            5
2                5
execise          5
spposed          5
n                5
fist             5
second           5
nah              5
’                5
ode              5
gg               5
call             5
thats            4
tho              4
sing             4
thee             4
vales            4
dont             4
need             4
wong             4
shold            4
data             4
good             4
qestion          4
model            4
